In [ ]:
%matplotlib inline
import numpy as np
import scipy.signal
import scipy.fft as sfft
import matplotlib.pylab as plt
from matplotlib import animation, patches, rc
import ipywidgets as widgets
import matplotlib as mpl
rc('animation', html='html5')
rc('savefig', dpi=80)
rc('figure', dpi=80)
from IPython.display import YouTubeVideo, HTML, Audio

# Diseño de sistemas y filtros FIR

## Sistema invariante al desplazamiento

Un sistema es invariante al desplazamiento  (*shift-invariant*) de su entrada si 

$$
f(x[n-m]) = y[n-m] 
$$

Es decir que 
- Aplicar un retardo en la entrada provoca un retardo equivalente en la salida
    - Un *blip* en la entrada produce un *blop* en la salida sin importar **cuando** ocurre el blip
- Las características del sistema no cambian con $n$
- Si $n$ representa el tiempo decimos que el sistema es **invariante en el tiempo**


***
Un sistema FIR es invariante al desplazamiento si los valores de $h$ son constantes para todo $n$

$$
h[n] = \text{Cte} \quad \forall n
$$

***

In [ ]:
fig, ax = plt.subplots(2, figsize=(8, 5), tight_layout=True)
n = np.arange(0, 500)
l0 = ax[0].plot(n, np.zeros_like(n)); 
ax[0].set_ylim([-0.1, 1.1]); ax[0].set_title('Input')
l1 = ax[1].plot(n, np.zeros_like(n));

def update(m):
    x = 0.5 + 0.5*scipy.signal.square((n-m)/(2.*np.pi*5), duty=0.3)      
    ax[1].set_ylim([-0.1, 1.1]); ax[1].set_title('Output')
    L = 50; h = np.ones(shape=(L, ))
    # h = 1.+ np.cos(2.0*np.pi*2.*(n-m)/len(n))
    h = h/np.sum(np.absolute(h))
    l0[0].set_ydata(x); l1[0].set_ydata(scipy.signal.convolve(x, h, mode='same'))
    return ()

anim = animation.FuncAnimation(fig, update, frames=500, interval=200, blit=True);

## Respuesta al impulso de un sistema


Sea el impulso unitario o delta de Kronecker

$$
\delta[n-m] = \begin{cases} 1 & n=m \\ 0 & n \neq m \end{cases}
$$

La **respuesta al impulso de un sistema discreto** es la salida obtenida cuando la entrada es un impulso unitario


Para un sistema FIR arbitrario tenemos

$$
y[n]|_{x=\delta} = (h * \delta)[n] = \sum_{j=0}^L h[j] \delta[n-j] = \begin{cases} h[n] & n \in [0, L] \\ 0 & \text{en otro caso} \end{cases} \\
$$

es decir que la respuesta al impulso:

-  tiene **una duración finita y luego decae a zero**
-  recupera los coeficientes $h[j]$ del sistema

En un sistema causal se tiene que $h[n] = 0 \quad \forall n < 0$

Llamamos **soporte** del sistema a todos aquellos valores de $n$ tal que  $h[n] \neq 0$



### Ejemplo

Para el sistema reverberante

$$
y[n] = x[n] + A x[n-m]
$$

la respuesta al impulso es

$$
y[n] = \delta[n] + A \delta[n-m] = \begin{cases} 1 & n=0\\ A& n=m \\ 0 & \text{en otro caso} \end{cases}
$$

La respuesta al impulse me permite a recuperar los coeficientes del sistema en caso de que no los conociera

## Sistema LTI y respuesta en frecuencia de un sistema

Los sistemas LTI (*linear time-invariant*) cumplen 

- linealidad: homogeneo y aditivo
- invarianza al desplazamiento de la entrada

y se expresan como

$$
y[n] = (h * x)[n] = \sum_j h[j] x[n-j]
$$
- h es la respuesta al impulso
- h guarda los coeficientes del sistema

***
Por propiedad de la DFT sabemos que un sistema LTI cumple

$$
\begin{align}
\text{DFT}_N [y[n]] & = \text{DFT}_N [(h * x)[n]] \nonumber \\
\text{DFT}_N [y[n]] & = \text{DFT}_N [h[n]]  \text{DFT}_N [x[n]] \nonumber \\
Y[k] &= H[k] X[k] , \nonumber
\end{align}
$$

donde convertimos la convolución temporal en una multiplicación frecuencial


- Llamamos a $H[k]$ la **respuesta en frecuencia del sistema** 
- La respuesta en frecuencia es la transformada de Fourier de la respuesta al impulso

***

Los sistemas LTI tienen intepretación directa en el dominio del frecuencia

***

En la próxima lección usaremos sistemas LTI para diseñar **filtros**

## Filtros digitales

Un **filtro** es un sistema cuyo objetivo es reducir o resaltar un cierto aspecto de una señal

Por ejemplo

- Disminuir el nivel de ruido
- Separar señales mezcladas
- Ecualizar
- Restaurar (eliminar desenfoque o artefactos de grabación)

Llamamos filtro digital a los filtros aplicados a señales digitales

Hablamos de "señal filtrada" para referirnos a la salida del filtro

A continuación estudiaremos filtros LTI, más adelante extenderemos a filtros adaptivos (no-TI) y filtros no-lineales (no-L)

***

El **filtro LTI** se puede estudiar en frecuencia usando 

$$
Y[k] = H[k] X[k] ,
$$

donde $H[k]$ es la DFT del filtro (respuesta en frecuencia)

- El **filtro LTI** actua como una máscara que modifica el espectro de la entrada
- Solo puede acentuar, atenuar o remover ciertas frecuencias pero **nunca crear nuevas frecuencias**

Consideremos los siguienes filtros (máscaras) ideales

<img src="../images/ideal_filters.gif">

***

## Diseño de un filtro FIR: Método de la ventana


Recordemos, un filtro FIR 

$$
\begin{align}
y[n] &= h[0] x[n] + h[1] x[n-1] + h[2] x[n-2] + \ldots + h[L] x[n-L] \nonumber \\
&= \sum_{j=0}^{L} h[j] x[n-j] \nonumber \\
&= (h* x)[n] \nonumber 
\end{align}
$$

está definido por su orden (L) y sus coeficientes $h[j], j=0,\ldots, L$
***
Diseñar un filtro consisten en encontrar L y los valores de los coeficientes
***
Podemos diseñar un filtro FIR siguiendo estos pasos

1. Especificar una **respuesta en frecuencia** ideal $H_d[k]$
1. Usar la transformada de Fourier inversa para obtener la **respuesta al impulso ideal** $h_d[n]$
1. Truncar la respuesta al impulso ideal usando **una ventana** tal que $h[n] = h_d[n] w[n]$
1. $h[n]$ nos da los coeficientes del filtro FIR y w[n] nos da el largo del filtro

La ventana $w[n]$ puede ser cualquiera de las funciones vistas en la unidad anterior, por ejemplo una ventana rectangular

$$
w[n] = \begin{cases} 1 & n \leq L \\ 0 & n > L \end{cases}
$$

o la ventana de Hann

$$
w[n] = 0.5 - 0.5 \cos \left( \frac{2\pi n}{L-1} \right)
$$



***

### Ejemplo: Filtro pasa bajo (LPF)

Un filtro pasa bajo es aquel que sólo deja pasar las **bajas** frecuencias

Sus usos son:
- Recuperar una tendencia o comportamiento lento en la señal
- Eliminar el ruido aditivo

Diseñemos un filtro que elimine todas las frecuencias mayores a $f_c$ [Hz] de una señal $x[n]$ muestreada con frecuencia $F_s$. 

1. Propongamos la siguiente respuesta en frecuencia ideal que solo deja pasar las frecuencias menores a $f_c$
$$
\begin{align}
H_d(\omega) &= \begin{cases} K & |f| < f_c\\ 0 & |f| > f_c \end{cases} \nonumber \\
&= K \text{rect}(f/f_c) \nonumber 
\end{align}
$$

1. Obtenemos su transformada de Fourier inversa
$$
\begin{align}
h_d(t) &= K \int_{-f_c}^{f_c} e^{j 2 \pi f t} df \nonumber \\
& = \frac{2j K f_c}{2 j \pi f_c t} \sin(2 \pi f_c t) = 2 K f_c \text{sinc}(2 \pi f_c t) \nonumber 
\end{align}
$$
que es una función infinitamente larga

1. La versión discreta sería
$$
h_d[n] = 2 K f_c\text{sinc}(2 \pi f_c n/ F_s) 
$$

1. Para obtener una respuesta al impulso finita multiplicamos por una ventana de largo finito

$$
h[n] = 2 K f_c \text{sinc}(2 \pi f_c n /F_s) \text{rect}(n/(L+1))
$$

que implica esto en la respuesta en frecuencia real?


In [ ]:
plt.close('all'); fig, ax = plt.subplots(5, 2, figsize=(8, 9))
n = np.arange(0, 100, step=1); 
f = fftpack.fftshift(fftpack.fftfreq(n=len(n), d=1))
np.random.seed(0); C = 5*np.exp(-0.5*(n[:, np.newaxis] - n[:, np.newaxis].T)**2/10**2)
x_clean = np.random.multivariate_normal(np.zeros_like(n), C)
x_clean -= np.mean(x_clean)
x = x_clean + 2*np.random.randn(len(n))
def update(fc, L, window='rect'):
    [axx.cla() for axx in ax.ravel()]
    # Señal limpia
    ax[0, 0].plot(n, x_clean); 
    ax[0, 1].plot(f, fftpack.fftshift(np.absolute(fftpack.fft(x_clean))))
    ax[0, 0].set_title("Tiempo"); ax[0, 1].set_title("Frecuencia")    
    # Señal contaminada
    ax[1, 0].plot(n, x); ylims = ax[1, 0].get_ylim(); ax[0, 0].set_ylim(ylims)
    X = fftpack.fft(x); 
    ax[1, 1].plot(f, fftpack.fftshift(np.absolute(X)))
    # Filtro ideal
    kc = int(len(n)*fc)
    Hd = np.zeros_like(n, dtype=np.float); Hd[:kc] = 1.; Hd[len(Hd)-kc+1:] = 1.
    ax[2, 1].plot(f, fftpack.fftshift(Hd))
    hd = np.real(fftpack.ifftshift(fftpack.ifft(Hd)))
    ax[2, 0].plot(n, hd)
    # Filtro real
    w = np.zeros_like(n, dtype=np.float); 
    if window == 'rect':
        w[len(w)//2-L//2:len(w)//2+L//2+1] = 1.;
    elif window == 'hann':
        w[len(w)//2-L//2:len(w)//2+L//2+1] = scipy.signal.hann(L+1)
    h = w*hd; H = fftpack.fft(h); 
    print("NMSE entre H y Hd %0.4f" % (np.mean((np.absolute(H) - Hd)**2)/np.mean(Hd**2)))
    ax[3, 0].plot(n, h);  ylims2 = ax[3, 0].get_ylim(); 
    ax[3, 0].plot(n, w, '--'); ax[3, 0].set_ylim(ylims2)
    ax[3, 1].plot(f, fftpack.fftshift(np.absolute(H)))
    # Señal filtrada
    xf = scipy.signal.convolve(x, h, mode='same')
    ax[4, 0].plot(n, np.real(xf)); 
    ax[4, 1].plot(f, fftpack.fftshift(np.absolute(fftpack.fft(xf)))); ax[4, 0].set_ylim(ylims)
    
interact(update, fc=FloatSlider_nice(min=0.01, max=0.5, step=0.01, value=0.1),
         L=SelectionSlider_nice(options=[4, 6, 10, 20, 40, 60, 80], value=60),
         window=SelectionSlider_nice(options=['rect', 'hann']));

- La respuesta en frecuencia "ideal" $H_d[k]$ es plana y tiene discontinuidades
- La respuesta en frecuencia "real" $H[k]$ busca aproximar a $H_d[k]$
- Observando $H[k]$ notamos que
    - No tiene zonas perfectamente planas (ni es perfectamente zero)
    - No tiene discontinuidades fuertes como el caso ideal
- Esto se debe al recorte que hacemos con la ventana

<img src="../images/system-real-filter.png" width="500">

- Podemos usar una ventana suave (Hann) para disminuir los *ripples* al costo de hacer más lenta las transiciones
***
El diseño del filtro está dado entonces por su
- **Función:** Definida por la respuesta en frecuencia ideal $|H_d[k]| \angle H_d[k]$ 
    - Podemos definir la magnitud o el ángulo dependiendo de la aplicación
- **Fidelidad:** El error tolerable entre la respuesta en frecuencia ideal y la real
***

***

### Ejemplo: Filtro pasa alto (HPF)

Un filtro pasa alto es aquel que sólo deja pasar las **altas** frecuencias

Sus usos son:
- Identificar cambios/detalles, es decir comportamientos rápidos en una señal
- Eliminar tendencias

Diseñemos un filtro que elimine todas las frecuencias menores a $f_c$ [Hz] de una señal $x[n]$ muestreada con frecuencia $F_s$. 

1. Propongamos la siguiente respuesta en frecuencia ideal que solo deja pasar las frecuencias **mayores** a $f_c$
$$
H_d(\omega) = \begin{cases} K & |f| \geq f_c\\ 0 & |f| < f_c \end{cases} 
$$

1. Obtenemos su transformada de Fourier inversa
$$
\begin{align}
h_d(t) &= K \int_{-\infty}^{-f_c} e^{j 2 \pi f t} df  + K \int_{f_c}^{\infty} e^{j 2 \pi f t} df\nonumber \\
&= K \int_{-\infty}^{\infty} e^{j 2 \pi f t} df  - K \int_{-f_c}^{f_c} e^{j 2 \pi f t} df\nonumber \\
& = K \delta(t) - 2 K f_c \text{sinc}(2 \pi f_c t) \nonumber 
\end{align}
$$

1. La versión discreta sería
$$
h_d[n] = K (\delta[n] - 2 F_c\text{sinc}(2 \pi f_c n/ F_s) )
$$

1. Finalmente la hacemos finita multiplicando por una ventana de largo finito
$$
h[n] = K (\delta[n] - 2 f_c\text{sinc}(2 \pi f_c n/ F_s) ) \text{rect}(n/(L+1))
$$

***
- Un filtro pasa alto se obtiene de forma trivial **restandole un filtro pasa bajo a un impulso unitario**
- Este "truco" se conoce como  **inversión espectral**


<img src="../images/system-hpf.png" width="400">


### Ejemplo: Filtro pasa banda (BPF) y rechaza banda (BRF)

- Como sus nombres lo indicam estos filtros 
    - Dejan pasar una cierta banda de frecuencia (BPF) o
    - Dejan pasar todas las frecuencias excepto una banda determinada (BRF)
- La banda de frecuencia está definida por sus frecuencias de corte mínima y máxima 
- Se pueden construir combinando el filtro pasa bajo y pasa alto

**Filtro pasa banda**: 
Para recuperar solo una cierta banda de frecuencia seguimos:
1. Definir frecuencias de corte $f_{c1} < f_{c2}$
- Construir un filtro pasa bajo con frecuencia de corte $f_{c2}$
- Convolucionar la señal con la respuesta al impulso resultante
- Construir un filtro pasa alto con frecuencia de corte $f_{c1}$
- Convolucionar la señal filtrada con la respuesta al impulso resultante

<img src="../images/system-bpf.png" width="400">

**Filtro rechazabanda**: 
Para eliminar una cierta banda de frecuencia seguimos:
1. Definir frecuencias de corte $f_{c1} > f_{c2}$
- Construir un filtro pasa bajo con frecuencia de corte $f_{c2}$
- Construir un filtro pasa alto con frecuencia de corte $f_{c1}$
- Sumar ambos las respuestas al impulso de ambos filtros
- Convolucionar la señal con la respuesta al impulso resultante

<img src="../images/system-rbf.png" width="400">

Respuesta en frecuencia de un filtro:

`scipy.signal.freqz(b, a=1, worN=512, whole=False, plot=None)`
- b (arreglo): coeficientes del numerador de $H[k]$
- a (arreglo): coeficientes del denominador de $H[k]$

In [ ]:
plt.close('all'); fig, ax = plt.subplots(4, figsize=(8, 8), tight_layout=True)

def low_pass_h(L, fc, fs):
    h = np.sinc(2*fc*np.arange(-L//2, L//2)/fs) 
    return h/np.sum(h) # ganancia unitaria K = 1/(2fc sum h)

L = 200+1; fc_lp, fc_hp, fs = 2, 3, 10
h_lp = low_pass_h(L, fc_lp, fs)
h_hp = -low_pass_h(L, fc_hp, fs); h_hp[L//2+1] += 1

h = h_lp
# h = h_hp + h_lp
# h = scipy.signal.convolve(h_hp, h_lp)

ax[0].plot(h); ax[0].set_ylabel("h[n]")
freq, response = scipy.signal.freqz(h)

ax[1].plot(0.5*fs*freq/np.pi, np.abs(response));
ax[2].plot(0.5*fs*freq/np.pi, np.unwrap(np.angle(response)));
freq, delay = scipy.signal.group_delay((h, 1))
ax[3].plot(0.5*fs*freq/np.pi, delay); ax[3].set_ylabel("Retardo")
ax[3].set_ylim([-50, 150])
ax[1].set_ylabel("|H[k]|"); ax[2].set_ylabel("angle H[k]"); 

In [ ]:
import librosa
data, sample_rate = librosa.load("../../data/nomekop.ogg", sr=44100)

fig, ax = plt.subplots(1, figsize=(8, 3))
freq, ttime, Sxx = scipy.signal.spectrogram(data, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax.pcolormesh(ttime, freq, 20*np.log10(Sxx+1e-4), cmap=plt.cm.magma); ax.set_ylim([0.0, 1e+4]);

Audio(data, rate=int(sample_rate), normalize=False)

`scipy.signal.firwin(numtaps, cutoff, width=None, window='hamming', pass_zero=True, scale=True, nyq=None, fs=None)`
- numtaps (entero): Largo del filtro 
- cutoff (arreglo flotante): frecuencias de corte
- window (string): tipo de ventana
- pass_zero (bool): pasa bajo o pasa alto
- fs (flotante): Frecuencia de muestreo

In [ ]:
h = scipy.signal.firwin(numtaps=1000+1, cutoff=[200, 500], 
                        pass_zero=False, window='hann',
                        fs=sample_rate)

fig = plt.figure(figsize=(8, 6), tight_layout=True)
ax1 = plt.subplot2grid((2, 2 ), (0, 0)); ax1.set_title("Respuesta al impulso")
ax2 = plt.subplot2grid((2, 2 ), (0, 1)); ax2.set_title("Respuesta en frecuencia")
ax3 = plt.subplot2grid((2, 2 ), (1, 0), colspan=2, rowspan=1)

freq, response = scipy.signal.freqz(h)
ax1.plot(h); ax2.semilogy(0.5*sample_rate*freq/np.pi, np.abs(response));
data_filt = scipy.signal.convolve(data, h)
freq, ttime, Sxx = scipy.signal.spectrogram(data_filt, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax3.pcolormesh(ttime, freq, 20*np.log10(Sxx+1e-4), cmap=plt.cm.magma); ax3.set_ylim([0.0, 1e+4]);

Audio(data_filt, rate=int(sample_rate), normalize=False)

## Filtro FIR con respuesta en frecuencia arbitraria

Consideremos un filtro FIR 
- De orden $L$ (par) con $L+1$ coeficientes 
- Centrado en el origen (cero fase)
- Con respuesta al impulso simétrica: $h_{n} = h_{-n}$

Podemos escribir su respuesta en frecuencia como 

$$
H[k] = \sum_{n=-L/2}^{L/2} h[n] e^{-j \frac{2\pi}{N} k n} = h[0] + \sum_{n=1}^{L/2} h[n]\cos \left(\frac{2\pi}{N} k n\right)
$$
que para $k=0, \ldots, N-1$ en forma matricial es

$$ 
\begin{align} 
\begin{pmatrix} H[0] \\ H[1] \\ H[2] \\ \vdots \\ H[N-1] \\ \end{pmatrix} 
&= 
\begin{pmatrix} 
1 & 1 & 1 & \cdots & 1 \\ 
1 & \cos \left(\frac{2\pi}{N} \right) & \cos \left(\frac{2\pi}{N} 2\right) & \cdots & \cos \left(\frac{2\pi}{N} L/2\right) \\ 
1 & \cos \left(\frac{2\pi}{N} 2 \right) & \cos \left(\frac{2\pi}{N} 4 \right) & \cdots & \cos \left(\frac{2\pi}{N} 2 L/2\right) \\ 
\vdots & \dots & \dots & \ddots & \vdots \\ 
1 & \cos \left(\frac{2\pi}{N} (N-1)\right) & \cos \left(\frac{2\pi}{N} (N-1) 2\right) & \cdots & \cos \left(\frac{2\pi}{N} (N-1) L/2\right) \\ 
\end{pmatrix} 
\begin{pmatrix} h[0] \\ h[1] \\ h[2] \\ \vdots \\ h[L/2] \\ \end{pmatrix} \nonumber \\
H &= A h, \nonumber
\end{align} 
$$

donde $A \in \mathbb{R}^{NxL/2}$

Si especificamos $H$ podemos despejar $h$ pero no podemos invertir A ya que no es cuadrada.
***
Una alternativa es buscar $\hat h \approx h$ que minimice el error cuadrático medio

$$
\min_h \| H - Ah\|^2,
$$
derivando e igualando a cero obtenemos el sistema de **ecuaciones normales**

$$
\begin{align}
\frac{d}{dh} \| H - Ah\|^2 &= 0 \nonumber \\
- 2 A^T (H - Ah) &= 0 \nonumber \\
A^T A h &= A^T H \nonumber \\
\hat h &= (A^T A)^{-1} A^T H  \nonumber \\
\hat h &= A^{\dagger} H, \nonumber
\end{align}
$$

donde 
- $A^{\dagger} = (A^T A)^{-1} A^T$ se conoce como la pseudo-inversa de A
- $\hat h$ se conoce como el **estimador de mínimos cuadrados** de h

***

`scipy.signal.firls(numtaps, bands, desired, weight=None, nyq=None, fs=None)`

- numtaps (entero impar): número de coeficientes
- bands: secuencia creciente de frecuencias
- desired: secuencia creciente de ganancias
- fs (flotante): Frecuencia de muestreo

In [ ]:
# Estimador de minimos cuadrados para filtro FIR usando scipy.signal
h = scipy.signal.firls(numtaps=1000+1, 
                       bands=(0., 640, 650, 2340, 2350, 2750, 2760, sample_rate//2), 
                       desired=(5., 5., 0., 0., 1., 1., 0., 0.), fs=sample_rate)

plt.close('all'); fig = plt.figure(figsize=(8, 6))
ax1 = plt.subplot2grid((2, 2 ), (0, 0)); ax1.set_title("Respuesta al impulso")
ax2 = plt.subplot2grid((2, 2 ), (0, 1)); ax2.set_title("Respuesta en frecuencia")
ax3 = plt.subplot2grid((2, 2 ), (1, 0), colspan=2, rowspan=1)

freq, response = scipy.signal.freqz(h)
ax1.plot(h); ax2.semilogy(0.5*sample_rate*freq/np.pi, np.abs(response));
data_filt = scipy.signal.convolve(data, h)
freq, ttime, Sxx = scipy.signal.spectrogram(data_filt, fs=sample_rate, window=('tukey', 0.25), 
                                            nperseg=256, noverlap=None, detrend=False,
                                            return_onesided=True, scaling='density', mode='magnitude')
ax3.pcolormesh(ttime, freq, 20*np.log10(Sxx+1e-4), cmap=plt.cm.magma); ax3.set_ylim([0.0, 1e+4]);
Audio(data_filt, rate=int(sample_rate))